In [111]:
__author__ = "Philipp Kats"
__date__ = "2016_02_09"

import pandas as pd

In [108]:
def selfMatrix(l):
    '''all possible pairs of elements '''

    for l1 in l:
        for l2 in l:
            yield (l1, l2)


In [31]:
df = pd.read_csv('sintetic.csv', index_col=0)
df = df[pd.notnull(df.postalCode)]

## Create Frame

every user presented as a row, every zipcode - as a column.
if user never posted in the zipcode, cell will be filled with NaN

therefore, when we filter the DF by df[(pd.notnull(pd[zipcode1])) & (pd.notnull(pd[zipcode2]))] we garantee that this user was in both zipcodes and we can get the sum of filtered rows

In [104]:
# first, aggregate by user_id and postal code, getting the sum of tweets from given user in given zipcode
g = df[['user_id','postalCode','id']].groupby(['user_id', 'postalCode']).agg('count')

In [105]:
# now, unstack the frame, getting all zipcodes as columns

frame = g.unstack()
frame.columns = [int(x) for x in frame.columns.droplevel().tolist()]
frame.head()

,83,10001,10002,10003,10004,10005,10006,10007,10009,10010,...,11432,11433,11434,11435,11436,11691,11692,11693,11694,11697
user_id,,,,,,,,,,,,,,,,,,,,,
47,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
418,NaN,2,6,4,NaN,NaN,NaN,NaN,13,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
577,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
648,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
882,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [107]:
def getValues(df, zip1, zip2):
    '''returns summ of values for both zipcodes
    only from users who were at both zipcodes,
    as two (zip, value) tuples'''
    
    return df[(pd.notnull(frame[zip1]))&(pd.notnull(frame[zip2]) )][[zip1, zip2]].sum().iteritems()
    

In [103]:
# test
for x in getValues(frame, 83, 10001):
    print x

(83, 5097.0)
(10001, 8244.0)


### Now lets create matrix of pairs

In [26]:
zipcodes = df.postalCode.dropna().astype(int).unique().tolist()
len(zipcodes)

In [112]:
matrix = []


for pair in selfMatrix(zipcodes):
    v1, v2 = getValues(frame, pair[0], pair[1])
    matrix.append({'zip1':v1[0], 'zip1_value':v1[1], 
                   'zip2':v2[0], 'zip2_value':v2[1] })


matrixDF = pd.DataFrame(matrix)
matrixDF.head(3)

,zip1,zip1_value,zip2,zip2_value
0,10304,536,10304,536
1,10304,12,11228,3
2,10304,77,10310,295


In [115]:
len(matrixDF)

58564

In [114]:
matrixDF.to_csv('matrix.csv')